# Determine the smallest optimal number of features to include

In [ ]:
import pandas as pd
import numpy as np
from mrmr import mrmr_classif
from sklearn.impute import KNNImputer
import os
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score, f1_score, average_precision_score, accuracy_score, precision_score, recall_score, confusion_matrix
from sklearn.utils.class_weight import compute_class_weight
import seaborn as sns
import matplotlib.pyplot as plt

CASE = 'CASE'
ID = 'ID'
data_dir = 'data'
subdir = 'experiment'

def rm_cols(df):
    df = df.drop([c for c in df.columns if 'px_' in c], axis=1)
    df = df.drop([c for c in df.columns if 'dx_Z' in c], axis=1)
    return df


def prep(X_train, X_test, cohort):
    # get target
    y_train = pd.merge(X_train, cohort, on=ID)[CASE]
    y_test = pd.merge(X_test, cohort, on=ID)[CASE]

    # get cols with >60% present
    col_freq = (X_train.notna().sum() / X_train.shape[0])
    col_freq = col_freq[col_freq > 0.6]
    freq_cols = col_freq.index
    X_train = X_train[freq_cols]

    # drop extra columns
    cols_to_drop = set(X_test.columns) - set(X_train.columns)
    X_test = X_test.drop(cols_to_drop, axis=1)

    return X_train, X_test, y_train, y_test


def run_mrmr(X_train, X_test, y_train, k):
    mrmr_results = mrmr_classif(X=X_train, y=y_train, K=k, show_progress=False, return_scores=True, relevance='rf')
    feature_idx = mrmr_results[0]
    feature_relevance = mrmr_results[1]
    feature_redundancy = mrmr_results[2]
    X_train = X_train.loc[:, feature_idx]
    X_test = X_test.loc[:, feature_idx]

    return X_train, X_test


def impute(X_train, X_test, cols, n):

    def make_bool(df):
        cols = [c for c in df.columns if 'dx_' in c or 'px_' in c or 'rx_' in c]
        df[cols] = df[cols].fillna(0)
        df[cols] = df[cols].apply(lambda x: x.apply(lambda y: 1 if y > 1 else y))
        return df

    # impute dx, rx, and px variables with 0, replace values >1 with 1.
    X_train = make_bool(X_train)
    X_test = make_bool(X_test)

    imputer = KNNImputer(n_neighbors=n, weights='distance')
    X_train[cols] = imputer.fit_transform(X_train)
    X_test[cols] = imputer.transform(X_test)

    # if boolean variable, round to 0 or 1.
    binary_features = set(['SMOKING_STATUS', 'ALCOHOL_USE_STATUS', 'ILLICIT_DRUG_USE', 'CARDIAC_HX'])
    binary_cols = []
    for b in binary_features:
        for c in X_train.columns:
            if b in c:
                binary_cols.append(c)
    binary_col_idx = np.array([i for i, e in enumerate(cols) if e in binary_cols])
    if binary_col_idx.shape[0] > 0:
        X_train.iloc[:,binary_col_idx] = np.round(X_train.iloc[:,binary_col_idx])
        X_test.iloc[:,binary_col_idx] = np.round(X_test.iloc[:,binary_col_idx])

    return X_train, X_test

def process(X1_train, X1_test, X2_train, X2_test, cohort, k=20, n=5, suffixes=['','']):
    if X2_train is not None:
        X_train = pd.merge(X1_train, X2_train, on=ID, how='left', suffixes=suffixes) # left because a couple of the tables have too few or too many patients
        X_test = pd.merge(X1_test, X2_test, on=ID, how='left', suffixes=suffixes)
    else:
        X_train = X1_train
        X_test = X1_test
    X_train = rm_cols(X_train)
    X_test = rm_cols(X_test)
    X_train, X_test, y_train, y_test = prep(X_train, X_test, cohort)

    X_train_idx = X_train.index
    X_test_idx = X_test.index
    y_train_idx = y_train.index
    y_test_idx = y_test.index

    X_train, X_test = run_mrmr(
        X_train.reset_index(drop=True), 
        X_test.reset_index(drop=True), 
        y_train.reset_index(drop=True), 
        k
    )

    X_train, X_test = impute(X_train, X_test, X_train.columns, n)

    X_train.index = X_train_idx
    X_test.index = X_test_idx
    y_train.index = y_train_idx
    y_test.index = y_test_idx

    # print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

    return X_train, X_test, y_train, y_test

def save(X_train, X_test, y_train, y_test, data_dir, name):
    os.makedirs(f'{data_dir}/{name}', exist_ok=True)
    X_train.to_csv(f'{data_dir}/{name}/X_train.csv')
    X_test.to_csv(f'{data_dir}/{name}/X_test.csv')
    y_train.to_csv(f'{data_dir}/{name}/y_train.csv')
    y_test.to_csv(f'{data_dir}/{name}/y_test.csv')

def combine(X1, X2, y):
    X = pd.merge(X1, X2, left_index=True, right_index=True)
    y = pd.merge(X, y, left_index=True, right_index=True)['CASE']
    return X, y


def binary_eval(model, x_train, y_train, x_test, y_test):
    # evaluate binary
    def score(y_pred, y_pred_proba, y_label):
        auc = roc_auc_score(y_label, y_pred_proba)
        auprc = average_precision_score(y_label, y_pred_proba)
        f1 = f1_score(y_label, y_pred)
        acc = accuracy_score(y_label, y_pred)
        precision = precision_score(y_label, y_pred)
        recall = recall_score(y_label, y_pred)
        results = [auc, auprc, f1, acc, precision, recall]
        conf_mat = confusion_matrix(y_label, y_pred)

        return results, conf_mat

    y_pred_train = model.predict(x_train)
    y_pred_proba_train = model.predict_proba(x_train)[:,1]

    y_pred_test = model.predict(x_test)
    y_pred_proba_test = model.predict_proba(x_test)[:,1]

    results_train, conf_mat_train = score(y_pred_train, y_pred_proba_train, y_train)
    results_test, conf_mat_test = score(y_pred_test, y_pred_proba_test, y_test)

    eval_results = pd.DataFrame(
            [results_train, results_test], 
            columns=['auc', 'auprc', 'f1', 'acc', 'precision', 'recall'],
            index=['train', 'test'])

    # feature importance
    feature_scores = pd.DataFrame(model.feature_importances_, index=x_train.columns, columns=['importance']).sort_values(by='importance', ascending=False)

    return eval_results, [conf_mat_train,conf_mat_test], feature_scores

In [ ]:
# load data
cohort = pd.read_csv(f'{data_dir}/cohort.csv', index_col=0)

# latest
X_latest_train = pd.read_csv(f'{data_dir}/X_latest_train.csv', index_col=0)
X_latest_test = pd.read_csv(f'{data_dir}/X_latest_test.csv', index_col=0)

# demographics
S_train = pd.read_csv(f'{data_dir}/S_train.csv', index_col=0)
S_test = pd.read_csv(f'{data_dir}/S_test.csv', index_col=0)

# labs vitals stats
X_cont_stats_train = pd.read_csv(f'{data_dir}/X_cont_stats_train.csv', index_col=0)
X_cont_stats_test = pd.read_csv(f'{data_dir}/X_cont_stats_test.csv', index_col=0)

# Dx Rx one hot encoded
X_dx_rx_px_history_train = pd.read_csv(f'{data_dir}/X_dx_rx_px_history_train.csv', index_col=0)
X_dx_rx_px_history_test = pd.read_csv(f'{data_dir}/X_dx_rx_px_history_test.csv', index_col=0)

# Dx Rx phenotypes
pheno_dir = f'{data_dir}/phenotypes_50_dxrx_HALS-exact'
X_train_dxrx = pd.read_csv(f'{pheno_dir}/pheno_patient_membership_train.csv', index_col=0)
X_test_dxrx = pd.read_csv(f'{pheno_dir}/pheno_patient_membership_test.csv', index_col=0)

# labs vitals phenotypes
pheno_dir = f'{data_dir}/phenotypes_30_lv_HALS-exact'
X_train_lv = pd.read_csv(f'{pheno_dir}/pheno_patient_membership_train.csv', index_col=0)
X_test_lv = pd.read_csv(f'{pheno_dir}/pheno_patient_membership_test.csv', index_col=0)

In [ ]:
X_train_dxrx.columns = [f'{c}_dxrx' for c in X_train_dxrx.columns]
X_test_dxrx.columns = [f'{c}_dxrx' for c in X_test_dxrx.columns]
X_train_lv.columns = [f'{c}_lv' for c in X_train_lv.columns]
X_test_lv.columns = [f'{c}_lv' for c in X_test_lv.columns]

## Compute feature importance/relevance score in mRMR

In [ ]:
suffixes = None
X1_train = X_train_dxrx
X1_test = X_test_dxrx
X2_train = X_train_lv
X2_test = X_test_lv


X_train = pd.merge(X1_train, X2_train, on=ID, how='left', suffixes=suffixes) # left because a couple of the tables have too few or too many patients
X_test = pd.merge(X1_test, X2_test, on=ID, how='left', suffixes=suffixes)
X_train = rm_cols(X_train)
X_test = rm_cols(X_test)
X_train, X_test, y_train, y_test = prep(X_train, X_test, cohort)

mrmr_results = mrmr_classif(X=X_train, y=y_train, K=10, show_progress=False, return_scores=True, relevance='rf')
feature_idx = mrmr_results[0]
feature_relevance = mrmr_results[1]
feature_redundancy = mrmr_results[2]

In [ ]:
class_weights = compute_class_weight(class_weight='balanced', classes=np.unique(y_train), y=y_train.values.reshape(-1))
class_weights = dict(zip(np.unique(np.unique(y_train)), class_weights))

X_train, X_valid, y_train, y_valid = train_test_split(X_train[feature_idx], y_train, test_size=0.2, random_state=0) # split training set into train and valid

results = []
for i in range(3):
    model = RandomForestClassifier(n_estimators=500, class_weight=class_weights, max_depth=5) # train model
    model.fit(X_train, y_train)
    eval_results, conf_mat, feature_scores = binary_eval(model, X_train, y_train, X_valid, y_valid) # evaluate model
    eval_results['i'] = i
    results.append(eval_results)

results = pd.concat(results)
results = results.reset_index()

### Latest + demo

In [ ]:
sns.lineplot(x=range(feature_relevance.values.shape[0]), y=feature_relevance.sort_values(ascending=False).values)
plt.ylabel('Feature Relevance')
plt.xlabel('Feature Rank')
plt.title('Feature Relevance (mRMR, kolmogorov-smirnov)')

In [ ]:
sns.lineplot(x=range(feature_relevance.values.shape[0]), y=feature_relevance.sort_values(ascending=False).values)
plt.ylabel('Feature Relevance')
plt.xlabel('Feature Rank')
plt.title('Feature Relevance (mRMR, f-statistic)')

In [ ]:
sns.lineplot(x=range(feature_relevance.values.shape[0]), y=feature_relevance.sort_values(ascending=False).values)
plt.ylabel('Feature Relevance')
plt.xlabel('Feature Rank')
plt.title('Feature Relevance (mRMR, RF)')

### Summary Statistics

In [ ]:
sns.lineplot(x=range(feature_relevance.values.shape[0]), y=feature_relevance.sort_values(ascending=False).values)
plt.ylabel('Feature Relevance')
plt.xlabel('Feature Rank')
plt.title('Feature Relevance (mRMR, kolmogorov-smirnov)')

In [ ]:
sns.lineplot(x=range(feature_relevance.values.shape[0]), y=feature_relevance.sort_values(ascending=False).values)
plt.ylabel('Feature Relevance')
plt.xlabel('Feature Rank')
plt.title('Feature Relevance (mRMR, f-statistic)')

In [ ]:
sns.lineplot(x=range(feature_relevance.values.shape[0]), y=feature_relevance.sort_values(ascending=False).values)
plt.ylabel('Feature Relevance')
plt.xlabel('Feature Rank')
plt.title('Feature Relevance (mRMR, RF)')

### Phenotypes

In [ ]:
sns.lineplot(x=range(feature_relevance.values.shape[0]), y=feature_relevance.sort_values(ascending=False).values)
plt.ylabel('Feature Relevance')
plt.xlabel('Feature Rank')
plt.title('Feature Relevance (mRMR, kolmogorov-smirnov)')

In [ ]:
sns.lineplot(x=range(feature_relevance.values.shape[0]), y=feature_relevance.sort_values(ascending=False).values)
plt.ylabel('Feature Relevance')
plt.xlabel('Feature Rank')
plt.title('Feature Relevance (mRMR, F-statistic)')

In [ ]:
sns.lineplot(x=range(feature_relevance.values.shape[0]), y=feature_relevance.sort_values(ascending=False).values)
plt.ylabel('Feature Relevance')
plt.xlabel('Feature Rank')
plt.title('Feature Relevance (mRMR, RF)')

## Compute performance of different number of features

In [ ]:
results_1 = []

for k in range(10,81,10):
    X_train, _, y_train, _ = process(X_train_dxrx, X_test_dxrx, X_train_lv, X_test_lv, cohort, suffixes=['_dxrx', '_lv'], k=k) # process data, mRMR, impute
    # X_train, _, y_train, _ = process(X_latest_train, X_latest_test, S_train, S_test, cohort, k=k)
    # X_train, _, y_train, _ = process(X_dx_rx_px_history_train, X_dx_rx_px_history_test, X_cont_stats_train, X_cont_stats_test, cohort, k=k)
    X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size=0.2, random_state=0) # split training set into train and valid

    # compute class weights
    class_weights = compute_class_weight(class_weight='balanced', classes=np.unique(y_train), y=y_train.values.reshape(-1))
    class_weights = dict(zip(np.unique(np.unique(y_train)), class_weights))

    for i in range(3):
        model = RandomForestClassifier(n_estimators=500, class_weight=class_weights, max_depth=5) # train model
        model.fit(X_train, y_train)
        eval_results, conf_mat, feature_scores = binary_eval(model, X_train, y_train, X_valid, y_valid) # evaluate model
        eval_results['k'] = k
        eval_results['i'] = i
        results_1.append(eval_results)

results_1 = pd.concat(results_1)
results_1 = results_1.reset_index()
results_1['index'] = results_1['index'].str.replace('test','valid')

In [ ]:
results_1 = results_1.rename(columns={
    'f1' : 'F1',
    'auc' : 'AUROC',
    'auprc' : 'AUPRC',
    'k' : 'Number of Features'
    })

In [ ]:
sns.set_theme(style="whitegrid")
sns.set_context("notebook", font_scale=1.5)
g = sns.lineplot(data=results_1, x='Number of Features', y='AUROC', hue='index')
g.figure.tight_layout()
g.legend_.remove()

In [ ]:
metrics = ['auc', 'auprc', 'f1', 'acc', 'precision', 'recall']

# Create a figure and 6 subplots, arranged in a 2x3 grid
fig, axes = plt.subplots(nrows=2, ncols=3, figsize=(20, 10))  # Adjust the size as needed

# Flatten the 2D array to 1D for iteration
axes = axes.flatten()

for ax, metric in zip(axes, metrics):
    sns.lineplot(data=results_1, x='k', y=metric, hue='index', ax=ax)
    ax.legend(title='Type')
    ax.set_title(metric)

plt.tight_layout()
fig.suptitle('RF performance with different number of phenotype features (k) selected by mRMR (mean and 95% conf int of 3 replicates)', fontsize=16, y=1.02)
plt.show()

In [ ]:
results_2 = []

for k in range(10,81,10):
    # X_train, _, y_train, _ = process(X_train_dxrx, X_test_dxrx, X_train_lv, X_test_lv, cohort, suffixes=['_dxrx', '_lv'], k=k) # process data, mRMR, impute
    X_train, _, y_train, _ = process(X_latest_train, X_latest_test, S_train, S_test, cohort, k=k)
    # X_train, _, y_train, _ = process(X_dx_rx_px_history_train, X_dx_rx_px_history_test, X_cont_stats_train, X_cont_stats_test, cohort, k=k)
    X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size=0.2, random_state=0) # split training set into train and valid

    # compute class weights
    class_weights = compute_class_weight(class_weight='balanced', classes=np.unique(y_train), y=y_train.values.reshape(-1))
    class_weights = dict(zip(np.unique(np.unique(y_train)), class_weights))

    for i in range(3):
        model = RandomForestClassifier(n_estimators=500, class_weight=class_weights, max_depth=5) # train model
        model.fit(X_train, y_train)
        eval_results, conf_mat, feature_scores = binary_eval(model, X_train, y_train, X_valid, y_valid) # evaluate model
        eval_results['k'] = k
        eval_results['i'] = i
        results_2.append(eval_results)

results_2 = pd.concat(results_2)
results_2 = results_2.reset_index()
results_2['index'] = results_2['index'].str.replace('test','valid')

In [ ]:
results_2 = results_2.rename(columns={
    'f1' : 'F1',
    'auc' : 'AUROC',
    'auprc' : 'AUPRC',
    'k' : 'Number of Features'
    })

In [ ]:
sns.set_theme(style="whitegrid")
sns.set_context("notebook", font_scale=1.5)
g = sns.lineplot(data=results_2, x='Number of Features', y='F1', hue='index')
g.figure.tight_layout()
g.legend_.remove()

In [ ]:
metrics = ['auc', 'auprc', 'f1', 'acc', 'precision', 'recall']

# Create a figure and 6 subplots, arranged in a 2x3 grid
fig, axes = plt.subplots(nrows=2, ncols=3, figsize=(20, 10))  # Adjust the size as needed

# Flatten the 2D array to 1D for iteration
axes = axes.flatten()

for ax, metric in zip(axes, metrics):
    sns.lineplot(data=results_2, x='k', y=metric, hue='index', ax=ax)
    ax.legend(title='Type')
    ax.set_title(metric)

plt.tight_layout()
fig.suptitle('RF performance with different number of latest+demo features (k) selected by mRMR (mean and 95% conf int of 3 replicates)', fontsize=16, y=1.02)
plt.show()

In [ ]:
results_3 = []

for k in range(10,81,10):
    # X_train, _, y_train, _ = process(X_train_dxrx, X_test_dxrx, X_train_lv, X_test_lv, cohort, suffixes=['_dxrx', '_lv'], k=k) # process data, mRMR, impute
    # X_train, _, y_train, _ = process(X_latest_train, X_latest_test, S_train, S_test, cohort, k=k)
    X_train, _, y_train, _ = process(X_dx_rx_px_history_train, X_dx_rx_px_history_test, X_cont_stats_train, X_cont_stats_test, cohort, k=k)
    X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size=0.2, random_state=0) # split training set into train and valid

    # compute class weights
    class_weights = compute_class_weight(class_weight='balanced', classes=np.unique(y_train), y=y_train.values.reshape(-1))
    class_weights = dict(zip(np.unique(np.unique(y_train)), class_weights))

    for i in range(3):
        model = RandomForestClassifier(n_estimators=500, class_weight=class_weights, max_depth=5) # train model
        model.fit(X_train, y_train)
        eval_results, conf_mat, feature_scores = binary_eval(model, X_train, y_train, X_valid, y_valid) # evaluate model
        eval_results['k'] = k
        eval_results['i'] = i
        results_3.append(eval_results)

results_3 = pd.concat(results_3)
results_3 = results_3.reset_index()
results_3['index'] = results_3['index'].str.replace('test','valid')

In [ ]:
results_3 = results_3.rename(columns={
    'f1' : 'F1',
    'auc' : 'AUROC',
    'auprc' : 'AUPRC',
    'k' : 'Number of Features'
    })

In [ ]:
sns.set_theme(style="whitegrid")
sns.set_context("notebook", font_scale=1.5)
g = sns.lineplot(data=results_3, x='Number of Features', y='AUROC', hue='index')
g.figure.tight_layout()
g.legend_.remove()
plt.savefig('ami/ami_180d_pxFalse_stringent-matchTrue_cntrl2_v2/sum-stats_n-features_auroc.png', dpi=300, bbox_inches='tight')

In [ ]:
metrics = ['auc', 'auprc', 'f1', 'acc', 'precision', 'recall']

# Create a figure and 6 subplots, arranged in a 2x3 grid
fig, axes = plt.subplots(nrows=2, ncols=3, figsize=(20, 10))  # Adjust the size as needed

# Flatten the 2D array to 1D for iteration
axes = axes.flatten()

for ax, metric in zip(axes, metrics):
    sns.lineplot(data=results_3, x='k', y=metric, hue='index', ax=ax)
    ax.legend(title='Type')
    ax.set_title(metric)

plt.tight_layout()
fig.suptitle('RF performance with different number of summary stat features (k) selected by mRMR (mean and 95% conf int of 3 replicates)', fontsize=16, y=1.02)
plt.show()

In [ ]:
cohort = pd.read_csv('data/cohort.csv', index_col=0)

data_path = 'data/experiment/'

X_ld_train = pd.read_csv(f'{data_path}/20_latest+demo/X_train.csv', index_col=0)
X_ld_test = pd.read_csv(f'{data_path}/20_latest+demo/X_test.csv', index_col=0)
y_ld_train = pd.read_csv(f'{data_path}/20_latest+demo/y_train.csv', index_col=0)
y_ld_test = pd.read_csv(f'{data_path}/20_latest+demo/y_test.csv', index_col=0)

X_agg_train = pd.read_csv(f'{data_path}/30_aggregate/X_train.csv', index_col=0)
X_agg_test = pd.read_csv(f'{data_path}/30_aggregate/X_test.csv', index_col=0)

X_phe_train = pd.read_csv(f'{data_path}/30_phenotypes/X_train.csv', index_col=0)
X_phe_test = pd.read_csv(f'{data_path}/30_phenotypes/X_test.csv', index_col=0)


X_latest_demo_agg_train, y_latest_demo_agg_train = combine(X_ld_train, X_agg_train, y_ld_train)
X_latest_demo_agg_test, y_latest_demo_agg_test = combine(X_ld_test, X_agg_train, y_ld_test)
# X_latest_demo_agg_train, X_latest_demo_agg_test = run_mrmr(
#     X_latest_demo_agg_train.reset_index(drop=True), 
#     X_latest_demo_agg_test.reset_index(drop=True), 
#     y_latest_demo_agg_train.reset_index(drop=True), 
#     k)

X_latest_demo_phe_train, y_latest_demo_phe_train = combine(X_ld_train, X_phe_train, y_ld_train)
X_latest_demo_phe_test, y_latest_demo_phe_test = combine(X_ld_test, X_phe_test, y_ld_test)
# X_latest_demo_phe_train, X_latest_demo_phe_test = run_mrmr(
#     X_latest_demo_phe_train.reset_index(drop=True), 
#     X_latest_demo_phe_test.reset_index(drop=True), 
#     y_latest_demo_phe_train.reset_index(drop=True), 
#     k)


# combine all
X_all_train = pd.merge(X_ld_train, X_agg_train, left_index=True, right_index=True)
X_all_train = pd.merge(X_all_train, X_phe_train, left_index=True, right_index=True)
y_all_train = pd.merge(X_all_train, y_ld_train, left_index=True, right_index=True)['CASE']
X_all_test = pd.merge(X_ld_test, X_agg_test, left_index=True, right_index=True)
X_all_test = pd.merge(X_all_test, X_phe_test, left_index=True, right_index=True)
y_all_test = pd.merge(X_all_test, y_ld_test, left_index=True, right_index=True)['CASE']

In [ ]:
results_4 = []

for k in range(10,51,10):
    X_train, _, y_train, _ = process(X_ld_train, X_ld_test, X_agg_train, X_agg_test, cohort, k=k)
    X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size=0.2, random_state=0) # split training set into train and valid

    # compute class weights
    class_weights = compute_class_weight(class_weight='balanced', classes=np.unique(y_train), y=y_train.values.reshape(-1))
    class_weights = dict(zip(np.unique(np.unique(y_train)), class_weights))

    for i in range(3):
        model = RandomForestClassifier(n_estimators=500, class_weight=class_weights, max_depth=5) # train model
        model.fit(X_train, y_train)
        eval_results, conf_mat, feature_scores = binary_eval(model, X_train, y_train, X_valid, y_valid) # evaluate model
        eval_results['k'] = k
        eval_results['i'] = i
        results_4.append(eval_results)

results_4 = pd.concat(results_4)
results_4 = results_4.reset_index()
results_4['index'] = results_4['index'].str.replace('test','valid')

results_4 = results_4.rename(columns={
    'f1' : 'F1',
    'auc' : 'AUROC',
    'auprc' : 'AUPRC',
    'k' : 'Number of Features'
    })

In [ ]:
sns.set_theme(style="whitegrid")
sns.set_context("notebook", font_scale=1.5)
g = sns.lineplot(data=results_4, x='Number of Features', y='F1', hue='index')
g.figure.tight_layout()
g.legend_.remove()

In [ ]:
metrics = ['AUROC', 'AUPRC', 'F1', 'acc', 'precision', 'recall']

# Create a figure and 6 subplots, arranged in a 2x3 grid
fig, axes = plt.subplots(nrows=2, ncols=3, figsize=(20, 10))  # Adjust the size as needed

# Flatten the 2D array to 1D for iteration
axes = axes.flatten()

for ax, metric in zip(axes, metrics):
    sns.lineplot(data=results_4, x='Number of Features', y=metric, hue='index', ax=ax)
    ax.legend(title='Type')
    ax.set_title(metric)

plt.tight_layout()
fig.suptitle('RF performance with different number of features (k) selected by mRMR (mean and 95% conf int of 3 replicates)', fontsize=16, y=1.02)
plt.show()

In [ ]:
results_5 = []

for k in range(10,51,10):
    X_train, _, y_train, _ = process(X_ld_train, X_ld_test, X_phe_train, X_phe_test, cohort, k=k)
    X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size=0.2, random_state=0) # split training set into train and valid

    # compute class weights
    class_weights = compute_class_weight(class_weight='balanced', classes=np.unique(y_train), y=y_train.values.reshape(-1))
    class_weights = dict(zip(np.unique(np.unique(y_train)), class_weights))

    for i in range(3):
        model = RandomForestClassifier(n_estimators=500, class_weight=class_weights, max_depth=5) # train model
        model.fit(X_train, y_train)
        eval_results, conf_mat, feature_scores = binary_eval(model, X_train, y_train, X_valid, y_valid) # evaluate model
        eval_results['k'] = k
        eval_results['i'] = i
        results_5.append(eval_results)

results_5 = pd.concat(results_5)
results_5 = results_5.reset_index()
results_5['index'] = results_5['index'].str.replace('test','valid')

results_5 = results_5.rename(columns={
    'f1' : 'F1',
    'auc' : 'AUROC',
    'auprc' : 'AUPRC',
    'k' : 'Number of Features'
    })

In [ ]:
sns.set_theme(style="whitegrid")
sns.set_context("notebook", font_scale=1.5)
g = sns.lineplot(data=results_5, x='Number of Features', y='AUROC', hue='index')
g.figure.tight_layout()
g.legend_.remove()

In [ ]:
metrics = ['AUROC', 'AUPRC', 'F1', 'acc', 'precision', 'recall']

# Create a figure and 6 subplots, arranged in a 2x3 grid
fig, axes = plt.subplots(nrows=2, ncols=3, figsize=(20, 10))  # Adjust the size as needed

# Flatten the 2D array to 1D for iteration
axes = axes.flatten()

for ax, metric in zip(axes, metrics):
    sns.lineplot(data=results_5, x='Number of Features', y=metric, hue='index', ax=ax)
    ax.legend(title='Type')
    ax.set_title(metric)

plt.tight_layout()
fig.suptitle('RF performance with different number of features (k) selected by mRMR (mean and 95% conf int of 3 replicates)', fontsize=16, y=1.02)
plt.show()

In [ ]:
results_6 = []

for k in range(10,81,10):
    X_train, _, y_train, _ = process(X_all_train, X_all_test, None, None, cohort, k=k)
    X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size=0.2, random_state=0) # split training set into train and valid

    # compute class weights
    class_weights = compute_class_weight(class_weight='balanced', classes=np.unique(y_train), y=y_train.values.reshape(-1))
    class_weights = dict(zip(np.unique(np.unique(y_train)), class_weights))

    for i in range(3):
        model = RandomForestClassifier(n_estimators=500, class_weight=class_weights, max_depth=5) # train model
        model.fit(X_train, y_train)
        eval_results, conf_mat, feature_scores = binary_eval(model, X_train, y_train, X_valid, y_valid) # evaluate model
        eval_results['k'] = k
        eval_results['i'] = i
        results_6.append(eval_results)

results_6 = pd.concat(results_6)
results_6 = results_6.reset_index()
results_6['index'] = results_6['index'].str.replace('test','valid')

results_6 = results_6.rename(columns={
    'f1' : 'F1',
    'auc' : 'AUROC',
    'auprc' : 'AUPRC',
    'k' : 'Number of Features'
    })

In [ ]:
sns.set_theme(style="whitegrid")
sns.set_context("notebook", font_scale=1.5)
g = sns.lineplot(data=results_6, x='Number of Features', y='F1', hue='index')
g.figure.tight_layout()
g.legend_.remove()

In [ ]:
metrics = ['AUROC', 'AUPRC', 'F1', 'acc', 'precision', 'recall']

# Create a figure and 6 subplots, arranged in a 2x3 grid
fig, axes = plt.subplots(nrows=2, ncols=3, figsize=(20, 10))  # Adjust the size as needed

# Flatten the 2D array to 1D for iteration
axes = axes.flatten()

for ax, metric in zip(axes, metrics):
    sns.lineplot(data=results_6, x='Number of Features', y=metric, hue='index', ax=ax)
    ax.legend(title='Type')
    ax.set_title(metric)

plt.tight_layout()
fig.suptitle('RF performance with different number of features (k) selected by mRMR (mean and 95% conf int of 3 replicates)', fontsize=16, y=1.02)
plt.show()